## 2020707035 박시언 lab4
1. 데이터에서 pass : 100000 , fail : all 이렇게 하여 데이터를 구성합니다.
2. LABEL에 OHE를 진행합니다.
3. trsin_df, test_df를 label-stratify하게 나눕니다.
4. 미디언필터를 적용하고 augmentation_df또한 구성합니다. 이때 fail data에 대해서만 augmentation을 진행합니다.
5. depth가 깊지 않으면서 적절한 차원의 vector가 나오게 모델을 구성합니다. 이는 resnet기반이고 skip connection을 사용하여 optim-gradient-map을 안정화 시킵니다.
6. custom dataset을 사용하는데 stratify한 sampling을 통한 배치 구성을 통해 배치샘플러를 사용합니다.
7. test Valanced acc가 높게 유지시키길 원하기 때문에 한 번 epoch를 돌 때마다 test를 진행하여 earlystop을 진행합니다.

### conclution
- valance acc는 0.8로 매우 높게 나타납니다.
- 이는 다음과 같은 이유 때문에 그러하다 생각합니다.
- 1. 기존의 머신러닝 알고리즘으로는 모델이 너무 작고 또한 고차원의 데이터를 적절히 학습하지 못한 것 같습니다. 또한 웨이퍼 데이터의 불량은 "패턴"으로 판별하게 되는데 이는 "위치" 보다는 어떤곳이던 해당 모양의 패턴을 감지하는 것이 중요하다는 것을 의미합니다. 머신러닝 알고리즘은 이러한 2d이미지의 패턴을 찾는데 효율적이지 못했다 생각합니다.
- 2. cnn의 경우에는 conv layer를 거치면서 kernel 사이즈 만큼의 픽셀 정보가 압축되어 효과적으로 패턴의 정보를 학습할 수 있기에 훨씬 높은 성능을 유지할 수 있을 것이라 생각합니다.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import median_filter, rotate, shift
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [2]:
df = pd.read_pickle("/home/unsi/다운로드/LSWMD.pkl")  #우분투 -> /home/unsi/다운로드/LSWMD.pkl || 맥 -> /Users/unsi/Downloads/LSWMD.pkl

In [3]:
df.iloc[35]['failureType']

array([], shape=(0, 0), dtype=float64)

In [4]:
# 빈 리스트를 포함하는 행만 필터링하기
passed = df[df['failureType'].apply(lambda x: x.shape == (0, 0) or x[0][0] == 'none')]
passed['failureType'] = 8
eda_pass = passed.copy()
passed = passed.sample(n=100000, random_state=42)

/tmp/ipykernel_29958/4080980931.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passed['failureType'] = 8


In [5]:
# Filtering rows based on a nested condition in the 'failureType' column
faild = df[df['failureType'].apply(lambda x: len(x) > 0 and len(x[0]) > 0 and x[0][0] != 'none')]
faild

,waferMap,dieSize,lotName,waferIndex,trianTestLabel,failureType
19,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,20.0,[[Training]],[[Loc]]
36,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,12.0,[[Training]],[[Edge-Loc]]
37,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,13.0,[[Training]],[[Edge-Loc]]
38,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,14.0,[[Training]],[[Edge-Loc]]
39,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,15.0,[[Training]],[[Edge-Loc]]
...,...,...,...,...,...,...
811450,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 2,...",600.0,lot47542,21.0,[[Test]],[[Edge-Loc]]
811451,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,...",600.0,lot47542,22.0,[[Test]],[[Edge-Loc]]
811452,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 1,...",600.0,lot47542,23.0,[[Test]],[[Edge-Ring]]
811453,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1,...",600.0,lot47542,24.0,[[Test]],[[Edge-Loc]]


In [6]:
faild.columns

Index(['waferMap', 'dieSize', 'lotName', 'waferIndex', 'trianTestLabel',
       'failureType'],
      dtype='object')

In [7]:
class2idx = {
    'Loc': 0,
    'Edge-Loc': 1,
    'Center': 2,
    'Edge-Ring': 3,
    'Scratch': 4,
    'Random': 5,
    'Near-full': 6,
    'Donut': 7,
    'pass': 8
}

#failureType 인코딩 
faild['failureType'] = faild['failureType'].apply(lambda x: class2idx.get(x[0][0], -1))

# 결과 확인
faild


/tmp/ipykernel_29958/1474913133.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  faild['failureType'] = faild['failureType'].apply(lambda x: class2idx.get(x[0][0], -1))


,waferMap,dieSize,lotName,waferIndex,trianTestLabel,failureType
19,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,20.0,[[Training]],0
36,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,12.0,[[Training]],1
37,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,13.0,[[Training]],1
38,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,14.0,[[Training]],1
39,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,15.0,[[Training]],1
...,...,...,...,...,...,...
811450,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 2,...",600.0,lot47542,21.0,[[Test]],1
811451,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,...",600.0,lot47542,22.0,[[Test]],1
811452,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 1,...",600.0,lot47542,23.0,[[Test]],3
811453,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1,...",600.0,lot47542,24.0,[[Test]],1


In [8]:
df = pd.concat([faild, passed])
df


,waferMap,dieSize,lotName,waferIndex,trianTestLabel,failureType
19,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,20.0,[[Training]],0
36,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,12.0,[[Training]],1
37,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,13.0,[[Training]],1
38,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,14.0,[[Training]],1
39,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,15.0,[[Training]],1
...,...,...,...,...,...,...
334843,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",5532.0,lot20174,4.0,[],8
609203,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1,...",710.0,lot38319,13.0,[],8
244074,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 2,...",648.0,lot15275,11.0,[],8
379250,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2393.0,lot22590,20.0,[],8


In [9]:
import pandas as pd
import numpy as np
from scipy.ndimage import median_filter

def apply_median_filter(wafer_map, size=3):
    return median_filter(wafer_map, size=size)

# waferMap 열에 필터 적용
df['waferMap'] = df['waferMap'].apply(lambda x: apply_median_filter(x, size=3))


In [13]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd


encoder = OneHotEncoder(sparse_output=False)
encoded_failure_type = encoder.fit_transform(df[['failureType']])
encoded_df = pd.DataFrame(encoded_failure_type, columns=encoder.get_feature_names_out(['failureType']))
df = pd.concat([df.reset_index(drop=True), encoded_df], axis=1)


In [14]:
df

,waferMap,dieSize,lotName,waferIndex,trianTestLabel,failureType,failureType_0,failureType_1,failureType_2,failureType_3,failureType_4,failureType_5,failureType_6,failureType_7,failureType_8
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,20.0,[[Training]],0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,12.0,[[Training]],1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,13.0,[[Training]],1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,14.0,[[Training]],1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,15.0,[[Training]],1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125514,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",5532.0,lot20174,4.0,[],8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
125515,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 1,...",710.0,lot38319,13.0,[],8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
125516,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 1,...",648.0,lot15275,11.0,[],8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
125517,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2393.0,lot22590,20.0,[],8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [15]:
from sklearn.model_selection import train_test_split
train_df,test_df = train_test_split(df, test_size=0.2, random_state=123,stratify=df['failureType'])

In [16]:
train_df

,waferMap,dieSize,lotName,waferIndex,trianTestLabel,failureType,failureType_0,failureType_1,failureType_2,failureType_3,failureType_4,failureType_5,failureType_6,failureType_7,failureType_8
68443,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2393.0,lot22826,5.0,[],8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
37654,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",3344.0,lot16320,20.0,[],8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
98221,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,...",1080.0,lot23504,12.0,[],8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
88610,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1791.0,lot45680,21.0,[[Training]],8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
33357,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 1, 2, 1,...",533.0,lot8,8.0,[[Training]],8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22387,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2,...",1109.0,lot45529,2.0,[[Test]],5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
124937,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,...",1109.0,lot8557,6.0,[],8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
119807,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1513.0,lot34231,17.0,[],8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8982,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1658.0,lot16605,18.0,[[Training]],3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [17]:
test_df

,waferMap,dieSize,lotName,waferIndex,trianTestLabel,failureType,failureType_0,failureType_1,failureType_2,failureType_3,failureType_4,failureType_5,failureType_6,failureType_7,failureType_8
124081,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,...",516.0,lot42981,18.0,[[Training]],8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
105532,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,...",518.0,lot45025,16.0,[[Training]],8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
102024,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,...",516.0,lot42873,17.0,[[Training]],8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
113212,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,...",1060.0,lot47404,25.0,[[Test]],8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1691,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2,...",1075.0,lot2524,14.0,[[Training]],3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8024,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2096.0,lot15190,7.0,[[Training]],1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36851,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1513.0,lot37573,23.0,[],8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
96269,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1724.0,lot22043,15.0,[],8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
25653,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1253.0,lot2697,9.0,[],8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [18]:
import numpy as np
import pandas as pd

def augment_wafer_map(wafer_map):
    augmented_maps = []
    
    # 원본 추가
    augmented_maps.append(wafer_map)

    # 회전 및 뒤집기 조합
    transformations = [
        lambda x: np.rot90(x, k=1),          # 90도 회전
        lambda x: np.rot90(x, k=2),          # 180도 회전
        lambda x: np.rot90(x, k=3),          # 270도 회전
        lambda x: np.flipud(x),              # 상하 뒤집기
        lambda x: np.fliplr(x),              # 좌우 뒤집기
    ]

    # 모든 변형 적용
    for transform in transformations:
        transformed_map = transform(wafer_map)
        augmented_maps.append(transformed_map)

    return augmented_maps

def augment_dataframe(df):
    augmented_data = []

    for _, row in df.iterrows():
        wafer_map = row['waferMap']
        die_size = row['dieSize']
        lot_name = row['lotName']
        wafer_index = row['waferIndex']
        train_test_label = row['trianTestLabel']
        failure_type = row['failureType']
        augmented_data.append(row.to_dict())
        # failure_type이 8이 아닐 경우에만 증강
        if failure_type != 8:
            augmented_wafer_maps = augment_wafer_map(wafer_map)
            for augmented_map in augmented_wafer_maps:
                augmented_data.append({
                    'waferMap': augmented_map,
                    'dieSize': die_size,
                    'lotName': lot_name,
                    'waferIndex': wafer_index,
                    'trianTestLabel': train_test_label,
                    'failureType': failure_type
                })


    # 증강된 데이터로 새로운 DataFrame 생성
    augmented_df = pd.DataFrame(augmented_data)
    return augmented_df

print("Original DataFrame shape:", train_df.shape)
train_df_aug = augment_dataframe(train_df)
print("Augmented DataFrame shape:", train_df_aug.shape)


Original DataFrame shape: (100415, 15)
Augmented DataFrame shape: (222905, 15)


In [19]:
train_df_aug['failureType'].value_counts()

failureType
8    80000
3    54208
1    29057
2    24045
0    20118
4     6685
5     4851
7     3108
6      833
Name: count, dtype: int64

In [20]:
train_df['failureType'].value_counts()

failureType
8    80000
3     7744
1     4151
2     3435
0     2874
4      955
5      693
7      444
6      119
Name: count, dtype: int64

In [21]:
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count

# 타겟 크기 설정
target_height, target_width = 60, 60

# 패딩 및 크기 조정 함수 (로그 변환 제거)
def pad_or_crop(image, target_shape=(60, 60)):
    """
    이미지의 크기가 target_shape와 동일하지 않은 경우, 잘라내거나 패딩을 추가하여 동일한 크기로 만듭니다.
    """
    current_shape = image.shape
    padded_image = np.zeros(target_shape, dtype=image.dtype)

    # 이미지의 크기가 목표 크기보다 작으면 중앙에 위치하도록 패딩
    padded_image[:min(current_shape[0], target_shape[0]), :min(current_shape[1], target_shape[1])] = \
        image[:min(current_shape[0], target_shape[0]), :min(current_shape[1], target_shape[1])]

    return padded_image

# 병렬 처리를 위한 함수
def parallel_apply(df, column):
    # 병렬 처리 수행
    with Pool(cpu_count()) as pool:
        transformed_maps = pool.map(pad_or_crop, df[column])
    
    # 데이터프레임에 변환된 웨이퍼 맵 추가
    df['paddedWaferMap'] = transformed_maps
    return df

# 각 데이터프레임에 병렬 처리 적용하여 결과 저장
train_df_filtered = parallel_apply(train_df, 'waferMap')
train_df_aug_filtered = parallel_apply(train_df_aug, 'waferMap')
test_df_filtered = parallel_apply(test_df, 'waferMap')


In [22]:
train_df_filtered['paddedWaferMap'][0]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [23]:
train_df_filtered.iloc[:, -10:-1]

,failureType_0,failureType_1,failureType_2,failureType_3,failureType_4,failureType_5,failureType_6,failureType_7,failureType_8
68443,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
37654,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
98221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
88610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
33357,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
22387,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
124937,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
119807,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8982,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [24]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
from tqdm import tqdm
from sklearn.metrics import balanced_accuracy_score

class WaferDataset(Dataset):
    def __init__(self, wafer_maps, labels):
        self.wafer_maps = wafer_maps.reset_index(drop=True)
        self.labels = labels.reset_index(drop=True)

    def __len__(self):
        return len(self.wafer_maps)

    def __getitem__(self, idx):
        wafer_map = self.wafer_maps[idx]
        wafer_map_tensor = torch.tensor(wafer_map, dtype=torch.float32).unsqueeze(0)
        label = torch.tensor(self.labels.iloc[idx].values, dtype=torch.float32)
        return wafer_map_tensor.to('cuda'), label.to('cuda')

# Define a consistent data processing pipeline for both augmented and non-augmented data
def create_dataloaders(df, batch_size=32):
    wafer_maps = df['paddedWaferMap']
    labels = df.iloc[:, -10:-1]  # Select one-hot encoded labels
    dataset = WaferDataset(wafer_maps, labels)

    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_ds, val_ds = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader

train_loader_noaug, val_loader_noaug = create_dataloaders(train_df_filtered)
train_loader_aug, val_loader_aug = create_dataloaders(train_df_aug_filtered)
test_loader, _ = create_dataloaders(test_df_filtered)

# Define the model
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return self.relu(out)

class MyEnhancedModel(nn.Module):
    def __init__(self, inch=1, num_classes=9):
        super(MyEnhancedModel, self).__init__()
        self.init_conv = nn.Sequential(
            nn.Conv2d(inch, 16, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True)
        )
        self.layer1 = ResidualBlock(16, 32, stride=2)
        self.layer2 = ResidualBlock(32, 64, stride=2)
        self.layer3 = ResidualBlock(64, 128, stride=2)
        self.layer4 = ResidualBlock(128, 256, stride=2)
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.init_conv(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.global_avg_pool(x)
        return self.fc(x)

# Training and validation function
def train_and_validate(model, train_loader, val_loader, epochs=50, patience=2, delta=0.001):
    model = model.to('cuda')
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    early_stopping_counter = 0
    best_val_loss = float('inf')

    for epoch in tqdm(range(epochs)):
        model.train()
        train_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to('cuda'), labels.argmax(dim=1).to('cuda')
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)

        train_loss /= len(train_loader.dataset)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to('cuda'), labels.argmax(dim=1).to('cuda')
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)

        val_loss /= len(val_loader.dataset)

        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

        if val_loss < best_val_loss - delta:
            best_val_loss = val_loss
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        if early_stopping_counter >= patience:
            print("Early stopping")
            break

# Initialize and train models for augmented and non-augmented data
model_no_aug = MyEnhancedModel()
train_and_validate(model_no_aug, train_loader_noaug, val_loader_noaug)

model_aug = MyEnhancedModel()
train_and_validate(model_aug, train_loader_aug, val_loader_aug)

# Evaluation function for calculating balanced accuracy
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to('cuda'), labels.argmax(dim=1).to('cuda')
            outputs = model(inputs)
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    balanced_acc = balanced_accuracy_score(all_labels, all_preds)
    print(f'Balanced Accuracy Score: {balanced_acc:.4f}')
    return balanced_acc

# Evaluate both models
print("Performance without augmentation:")
evaluate_model(model_no_aug, test_loader)

print("\nPerformance with augmentation:")
evaluate_model(model_aug, test_loader)


  2%|▏         | 1/50 [00:09<07:53,  9.67s/it]

Epoch 1, Train Loss: 0.3556, Validation Loss: 0.2818


  4%|▍         | 2/50 [00:19<07:37,  9.53s/it]

Epoch 2, Train Loss: 0.2770, Validation Loss: 0.2683


  6%|▌         | 3/50 [00:28<07:25,  9.48s/it]

...

Epoch 8, Train Loss: 0.2300, Validation Loss: 0.2599


 16%|█▌        | 8/50 [03:04<16:10, 23.11s/it]

Epoch 9, Train Loss: 0.2204, Validation Loss: 0.2683
Early stopping
Performance without augmentation:


Balanced Accuracy Score: 0.7763

Performance with augmentation:
Balanced Accuracy Score: 0.2113


0.21132012675348052

In [26]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import balanced_accuracy_score
from tqdm import tqdm

class WaferDataset(Dataset):
    def __init__(self, wafer_maps, labels):
        self.wafer_maps = wafer_maps.reset_index(drop=True)
        self.labels = labels.reset_index(drop=True)

    def __len__(self):
        return len(self.wafer_maps)

    def __getitem__(self, idx):
        wafer_map = self.wafer_maps[idx]
        wafer_map_tensor = torch.tensor(wafer_map, dtype=torch.float32).unsqueeze(0)
        label = torch.tensor(self.labels.iloc[idx].values, dtype=torch.float32)
        return wafer_map_tensor.to('cuda'), label.to('cuda')

def create_dataloaders(df, batch_size=32):
    wafer_maps = df['paddedWaferMap']
    labels = df.iloc[:, -10:-1]  # 원-핫 인코딩된 라벨 선택
    dataset = WaferDataset(wafer_maps, labels)
    
    # 클래스 가중치를 사용한 샘플링 적용
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels.values.argmax(axis=1)), 
                                         y=labels.values.argmax(axis=1))
    sample_weights = class_weights[labels.values.argmax(axis=1)]
    sampler = WeightedRandomSampler(sample_weights, len(sample_weights))

    data_loader = DataLoader(dataset, batch_size=batch_size, sampler=sampler)
    return data_loader

# 원본 데이터셋 로더 생성
train_loader_no_aug = create_dataloaders(train_df_filtered)
val_loader_no_aug = create_dataloaders(test_df_filtered)

# augmentation 데이터셋 로더 생성
train_loader_aug = create_dataloaders(train_df_aug_filtered)

# 모델 정의 및 학습

class MyEnhancedModel(nn.Module):
    def __init__(self, inch=1, num_classes=9):
        super(MyEnhancedModel, self).__init__()
        self.init_conv = nn.Sequential(
            nn.Conv2d(inch, 16, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True)
        )
        self.layer1 = ResidualBlock(16, 32, stride=2)
        self.layer2 = ResidualBlock(32, 64, stride=2)
        self.layer3 = ResidualBlock(64, 128, stride=2)
        self.layer4 = ResidualBlock(128, 256, stride=2)
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.init_conv(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.global_avg_pool(x)
        return self.fc(x)

# 평가 함수
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to('cuda'), labels.argmax(dim=1).to('cuda')
            outputs = model(inputs)
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    balanced_acc = balanced_accuracy_score(all_labels, all_preds)
    return balanced_acc

# 학습 및 early stopping 구현 함수
def train_and_evaluate_with_early_stopping(model, train_loader, val_loader, epochs=50, patience=5, delta=0.001):
    model = model.to('cuda')
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    best_val_acc = 0.0
    early_stopping_counter = 0

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to('cuda'), labels.argmax(dim=1).to('cuda')
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
        
        train_loss /= len(train_loader.dataset)

        # Validation 평가 및 early stopping 체크
        val_acc = evaluate_model(model, val_loader)
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Validation Balanced Acc: {val_acc:.4f}')
        
        # Early stopping 조건 체크
        if val_acc > best_val_acc + delta:
            best_val_acc = val_acc
            early_stopping_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')  # 최적 모델 저장
        else:
            early_stopping_counter += 1

        if early_stopping_counter >= patience:
            print("Early stopping triggered")
            break

    # 최적 모델 로드
    model.load_state_dict(torch.load('best_model.pth'))
    print(f"Training complete. Best Validation Balanced Accuracy: {best_val_acc:.4f}")

# 원본 데이터로 모델 학습 및 평가
model_no_aug = MyEnhancedModel()
print("Training and evaluating on original data with early stopping:")
train_and_evaluate_with_early_stopping(model_no_aug, train_loader_no_aug, val_loader_no_aug)

# augmentation 데이터로 모델 학습 및 평가
model_aug = MyEnhancedModel()
print("\nTraining and evaluating on augmented data with early stopping:")
train_and_evaluate_with_early_stopping(model_aug, train_loader_aug, val_loader_no_aug)


Training and evaluating on original data with early stopping:
Epoch 1/50, Train Loss: 0.5507, Validation Balanced Acc: 0.7923
Epoch 2/50, Train Loss: 0.2944, Validation Balanced Acc: 0.8085
Epoch 3/50, Train Loss: 0.2143, Validation Balanced Acc: 0.8068
Epoch 4/50, Train Loss: 0.1725, Validation Balanced Acc: 0.8214
Epoch 5/50, Train Loss: 0.1478, Validation Balanced Acc: 0.8148
Epoch 6/50, Train Loss: 0.1275, Validation Balanced Acc: 0.8191
Epoch 7/50, Train Loss: 0.1175, Validation Balanced Acc: 0.8155
Epoch 8/50, Train Loss: 0.1038, Validation Balanced Acc: 0.8298
Epoch 9/50, Train Loss: 0.0971, Validation Balanced Acc: 0.8181
Epoch 10/50, Train Loss: 0.0907, Validation Balanced Acc: 0.7983
Epoch 11/50, Train Loss: 0.0892, Validation Balanced Acc: 0.8294
Epoch 12/50, Train Loss: 0.0803, Validation Balanced Acc: 0.8083
Epoch 13/50, Train Loss: 0.0780, Validation Balanced Acc: 0.8047
Early stopping triggered
Training complete. Best Validation Balanced Accuracy: 0.8298

Training and ev

/tmp/ipykernel_29958/430022312.py:133: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Epoch 1/50, Train Loss: 0.4756, Validation Balanced Acc: 0.7659
Epoch 2/50, Train Loss: 0.2737, Validation Balanced Acc: 0.7806
Epoch 3/50, Train Loss: 0.2268, Validation Balanced Acc: 0.7756
Epoch 4/50, Train Loss: 0.2020, Validation Balanced Acc: 0.7792
Epoch 5/50, Train Loss: 0.1872, Validation Balanced Acc: 0.7479
Epoch 6/50, Train Loss: 0.1748, Validation Balanced Acc: 0.7647
Epoch 7/50, Train Loss: 0.1655, Validation Balanced Acc: 0.7521
Early stopping triggered
Training complete. Best Validation Balanced Accuracy: 0.7806


/tmp/ipykernel_29958/430022312.py:133: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))
